# MINST: 从0到1%:第一次提交

在我们学习深度学习的时候，往往会用最简单的MNIST数据集作为我们的开始。在这里我用kaggle上的 digit-recognizer为初学者作为讲解。
Kaggle传送门: [digit-recognizer](https://www.kaggle.com/c/digit-recognizer) 
我们目的并不是成为kaggle上的1%,我希望通过这篇文章，大家能对CV上的训练有一个最最简单的认识，并且对参加kaggle竞赛的过程有一个理解。

In [1]:
import torch
#本文我们使用的是pytorch 1.0.0
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
import pandas as pd
import numpy as np #numpy 是一个极其重要的库，据说75%的机器学习项目都用了numpy
# pytorch是个代码即文档的库，我建议如果你想搞懂pytoch的话，可以直接看代码

### 数据准备

首先我们对kaggle下载的数据解压，并处理，在这里我已经下好了，得到的就是test.csv, train.csv, sample_submission.csv三个文件。
接下来我们利用pytorch中的Dataset和DataLoader对数据进行处理。

In [2]:
trans_img = transforms.Compose([
    transforms.ToTensor()
])
#将numpy数组转为pytorch训练的tensor

打开从kaggle下载的train.csv，可以看到他给我们的文件其实就是一个label和一个一维化的图像

In [3]:
train_path = "train.csv"
try_data = pd.read_csv(train_path,skiprows = 0)
print(try_data.shape)

(42000, 785)


#### 继承Dataset

然后我们从pytorch的Dataset那里继承我们的MNISTDataset，我们把训练用的item从读入的1维的图像转为3维的。这里我们将一个图像转为[1,28,28]形状的，主要是适应pytorch自带的MNIST数据集，这是后话。

In [4]:
class MNISTDataset(Dataset):
    def __init__(self,csv_file,transform=None):
        data = pd.read_csv(csv_file,skiprows=0)
        self.X = np.array(data.iloc[:,1:]).reshape(-1,28,28,1).astype('float32')
        self.X /= 255
        self.y = np.array(data.iloc[:,0])
        del data
        self.transfrom = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        item = self.X[idx]
        label = torch.from_numpy(np.array(self.y[idx]))
        if(self.transfrom):
            item = self.transfrom(item)
        return (item,label)

In [5]:
batch_size = 50 #batch_size是一次训练读入的图片数量
kaggle_trainset = MNISTDataset(csv_file='train.csv',transform=trans_img)
print(kaggle_trainset.__getitem__(0)[0].shape)
kaggle_trainloader = DataLoader(dataset = kaggle_trainset,batch_size=batch_size,shuffle=True)
#shuffle 是训练集是否随机读取

torch.Size([1, 28, 28])


### 第一个model

通过DataLoader我们就获取到了一个适合训练的训练集了。然后我们就好建立我们的模型了。在这里我们使用的是著名的Lenet-5模型，很简单，大家可以百度一下，大致就是两个卷积两个池化最后来三个全连接。具体的模型可以自己看看教程或者直接看论文。

In [6]:
import torch.nn as nn
import torch.nn.functional as F
#pytorch 的网络层都在nn里面，我建议之间看pytorch的代码
#另外 pytorch的激活函数是在 nn.functional 里面
#最简单的lenet-5
class Lenet1(nn.Module):
    def __init__(self):
        super(Lenet1, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(6,16,5, stride= 1, padding=0)
        self.fc1 = nn.Linear(400,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self, x):
        out = F.max_pool2d(self.conv1(x),(2,2))
        out = F.max_pool2d(self.conv2(out),(2,2))
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

### 训练模型

接下来我们肯定要迫不及待地试验一下我们模型的结果啦。怎么看效果呢，这时候我们就能使用pytorch自带的MNIST数据集来验证。

In [7]:
from torchvision.datasets import MNIST
MNIST_vaildset = MNIST('./data', train=False,download = False ,transform=trans_img)
# 如果是第一次使用，需要将download改成True，等待的事件可能还有点久
MNIST_vaildloader = DataLoader(MNIST_vaildset,batch_size=batch_size,shuffle = True)
#使用pytorch自带的验证集

In [14]:
from torch.autograd import Variable
from torch import optim

def train(model,trainloader,optimizer,epoch,epoches):
    criterian = nn.CrossEntropyLoss(size_average=False)
    running_loss = 0.
    running_acc = 0.
    for (img, label) in trainloader:
        if torch.cuda.is_available():
            img = Variable(img).cuda()
            label = Variable(label).cuda()

        optimizer.zero_grad()
        output = model(img)
        loss = criterian(output, label)
        # backward
        loss.backward()
        optimizer.step()

        running_loss += loss.data
        _, predict = torch.max(output, 1)
        correct_num = (predict == label).sum()
        running_acc += correct_num.item()
    running_loss /= len(trainloader.dataset)
    running_acc /= len(trainloader.dataset)
    print("[%d/%d] Loss: %f, Acc: %f" % (epoch + 1, epoches, running_loss, 100 * running_acc))
    
def vaild(model,vaildloader):
    criterian = nn.CrossEntropyLoss(size_average=False)
    vaildloss = 0.
    vaildacc = 0.
    for (img, label) in MNIST_vaildloader:
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        output = model(img)
        loss = criterian(output, label)
        vaildloss += loss.data
        _, predict = torch.max(output, 1)
        num_correct = (predict == label).sum()
        vaildacc += num_correct.item()
    vaildloss /= len(vaildloader.dataset)
    vaildacc /= len(vaildloader.dataset)
    print("Test: Loss: %.5f, Acc: %.2f %%" % (vaildloss, 100 * vaildacc))

#### 开始训练

In [11]:
learning_rate = 1e-3
epoches = 20
lenet1 = Lenet1()
if torch.cuda.is_available():
    lenet1.cuda()
optimizer1 = optim.SGD(lenet1.parameters(), lr=learning_rate)
for epoch in range(epoches):
    train(model=lenet1,trainloader=kaggle_trainloader,optimizer=optimizer1,epoch=epoch,epoches=epoches)
    if (epoch+1)%10 == 0:
        vaild(model = lenet1,vaildloader=MNIST_vaildloader)

[1/20] Loss: 0.527811, Acc: 83.811905
[2/20] Loss: 0.127143, Acc: 96.119048
[3/20] Loss: 0.090400, Acc: 97.254762
[4/20] Loss: 0.073504, Acc: 97.766667
[5/20] Loss: 0.063877, Acc: 98.045238
[6/20] Loss: 0.058642, Acc: 98.150000
[7/20] Loss: 0.051914, Acc: 98.357143
[8/20] Loss: 0.047743, Acc: 98.445238
[9/20] Loss: 0.044504, Acc: 98.600000
[10/20] Loss: 0.041794, Acc: 98.697619


AttributeError: 'builtin_function_or_method' object has no attribute 'item'

### 改进模型

理论上来说，在20次的迭代后这个模型能达到99%的正确率，但是我们的目标可是成为~~海贼王的男人~~100%，那么就要改进一下啦。
通过从网上找资料，我发现了激活函数这个好东西，好像说这几年的深度学习能这么好，有很大的功劳是激活函数的不断发展。
那么我们不如就试一下把一些激活函数套上我们平凡的lenet模型。

In [15]:
learning_rate = 1e-3
epoches = 20
class Lenet2(nn.Module):
    def __init__(self):
        super(Lenet2, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(6,16,5, stride= 1, padding=0)
        self.fc1 = nn.Linear(400,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self, x):
        out = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        out = F.max_pool2d(F.relu(self.conv2(out)),(2,2))
        out = out.view(out.size(0),-1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
    

lenet2 = Lenet2()
if torch.cuda.is_available():
    lenet2.cuda()
optimizer2 = optim.SGD(lenet2.parameters(), lr=learning_rate)
for epoch in range(epoches):
    train(model=lenet2,trainloader=kaggle_trainloader,optimizer=optimizer2,epoch=epoch,epoches=epoches)
    if (epoch+1)%10 == 0:
        vaild(model = lenet2,vaildloader=MNIST_vaildloader)

[1/20] Loss: 0.856990, Acc: 71.078571
[2/20] Loss: 0.114097, Acc: 96.438095
[3/20] Loss: 0.079466, Acc: 97.440476
[4/20] Loss: 0.061130, Acc: 98.030952
[5/20] Loss: 0.048927, Acc: 98.423810
[6/20] Loss: 0.042337, Acc: 98.592857
[7/20] Loss: 0.036742, Acc: 98.773810
[8/20] Loss: 0.032797, Acc: 98.859524
[9/20] Loss: 0.027252, Acc: 99.140476
[10/20] Loss: 0.026037, Acc: 99.114286
Test: Loss: 0.02427, Acc: 99.24 %
[11/20] Loss: 0.020766, Acc: 99.319048
[12/20] Loss: 0.019176, Acc: 99.333333
[13/20] Loss: 0.016288, Acc: 99.438095
[14/20] Loss: 0.013642, Acc: 99.569048
[15/20] Loss: 0.012957, Acc: 99.597619
[16/20] Loss: 0.011243, Acc: 99.630952
[17/20] Loss: 0.010216, Acc: 99.671429
[18/20] Loss: 0.009779, Acc: 99.650000
[19/20] Loss: 0.006942, Acc: 99.819048
[20/20] Loss: 0.006559, Acc: 99.778571
Test: Loss: 0.01363, Acc: 99.66 %


从结果来看，稍微好了那么一点点，但是我们不满足啊，我们一定要继续努力，从而变得更~~秃~~强!

### 保存模型

In [16]:
torch.save(lenet2,"mnist.pkl")
#保存我们的lenet2模型

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Lenet2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### 创建测试集的Dataset

打开test.csv我们可以看到它和train.csv的数据是不一样的，因此我们以为test.csv也创建一个Dataset

In [17]:
class testdata(Dataset):
    def __init__(self,csv_file,transform = None):
        data = pd.read_csv(csv_file, skiprows=0)
        self.X = np.array(data).reshape(-1, 28, 28, 1).astype('float32')
        self.X /= 255
        del data
        self.transfrom = transform

    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        item = self.X[idx]
        if(self.transfrom):
            item = self.transfrom(item)
        return item

testdataset = testdata("test.csv",transform=trans_img)
testloader = DataLoader(dataset=testdataset,batch_size=batch_size,shuffle=False)

### 输出测试结果

除了两个数据集，kaggle给我们的还有一个sample_submission.csv,我们需要仿照这个格式提交我们的结果。

In [20]:
def test(model,dataloader,path):
    model.eval() #固定model的参数
    f = open(path,'w')
    f.write('ImageId,Label\n')
    i = 1
    for img in testloader:
        if torch.cuda.is_available():
            img = Variable(img).cuda()
        output = model(img)
        _, predict = torch.max(output, 1)
        for it in predict:
            f.write(str(i)+','+str(it.item())+'\n')
            i += 1
    f.close
    print("OK!")
model = torch.load("mnist.pkl")
test(model,testloader,'submit.csv')

### 提交模型

现在就能把得到的submit.csv交上去，然后得到第一个kaggle排名！
我们这里的迭代次数只有20次，可以通过更多的迭代得到更好的模型，但是更过的迭代的结果往往是过拟合，那么应该怎么解决呢？且听下回分解。